In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("seaborn")
from scipy.optimize import minimize

In [2]:
import RSI
import MACD

In [4]:
def optimal_strategy(parameters):
    
    start = "2004-01-01"
    end = "2020-06-30"
    symbol = "EURUSD=X"
    tc = 0.00007
    
    # SMA
    tester1 = RSI.RSIBacktester(symbol, int(parameters[0]), int(parameters[1]), int(parameters[2]), start, end, tc)
    tester1.test_strategy()
    
    # Bollinger
    tester2 = MACD.MACDBacktester(symbol,  int(parameters[3]),  int(parameters[4]), int(parameters[5]), start, end, tc)
    tester2.test_strategy()
    
    # Create comb
    comb = tester1.results.loc[:, ["returns", "position"]].copy()
    comb.rename(columns = {"position":"position_RSI"}, inplace = True)
    comb["position_MACD"] = tester2.results.position
    
    comb["position_comb"] = np.sign(comb.position_MACD + comb.position_RSI)
    
    # Backtest
    comb["strategy"] = comb["position_comb"].shift(1) * comb["returns"]
    comb.dropna(inplace=True)
    comb["trades"] = comb.position_comb.diff().fillna(0).abs()
    comb.strategy = comb.strategy - comb.trades * tc
    comb["creturns"] = comb["returns"].cumsum().apply(np.exp)
    comb["cstrategy"] = comb["strategy"].cumsum().apply(np.exp)
    
    return -comb["cstrategy"].iloc[-1]


In [5]:
optimal_strategy((14, 70, 30, 12, 26, 9))

-0.5459285070880692

In [6]:
bnds = ((10, 20), (60, 80), (20, 40), (5, 20), (20, 50), (5, 20))

In [7]:
start_par = (14, 70, 30, 12, 26, 9)

In [8]:
opts = minimize(optimal_strategy, start_par, method = "Powell" , bounds = bnds)

In [9]:
opts

   direc: array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ],
       [ 0.09675752, -0.39877323, -1.35269101, -1.21411122, -0.67820993,
        -1.42468948]])
     fun: -1.3368046714133892
 message: 'Optimization terminated successfully.'
    nfev: 566
     nit: 4
  status: 0
 success: True
       x: array([13.82309847, 72.19222662, 38.1494794 , 17.94404425, 35.4369478 ,
       16.40935294])

In [10]:
optimal_strategy(opts.x)

-1.3368046714133892